# Применение модели Helsinki-NLP/opus-mt-ru-en
# на параллельном корпусе парафраз RuAdapt
https://huggingface.co/Helsinki-NLP/opus-mt-ru-en

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/zlatoust_sentence_aligned_with_CATS.csv')
# df

,source,cos_sim,target,level
0,Ну что ж.,1.000000,Ну что ж.,b2
1,"— Ах, только не мужем, с простою усмешкой сказ...",0.863004,"— Ах, только не мужем, — с простою усмешкой ск...",b1
2,"Мне скоро шестьдесят, я старик, одинокий, ничт...",0.653318,"Ничего во мне нет хорошего, кроме этой любви к...",b2
3,Да!,0.093739,Обстановочка-то какова!,a2_b1
4,"Я вполне уверен, что вы раскаялись и раскаивае...",0.621839,"Я уверен, что вы раскаялись и будете содейство...",b1
...,...,...,...,...
24227,У нас в феврале масленица,0.960889,У нас в феврале масленица.,b1_c1
24228,— Разводиться мне или нет?,0.962737,— Разводиться мне или нет?,b1
24229,"Парусиновую куртку Петр Алексеевич сбросил, ру...",0.938603,"Парусиновую куртку Пётр Алексеевич сбросил, р...",b1_b2
24230,Ай!,1.000000,Ай!,b2


In [ ]:
import pandas as pd
new_df = pd.read_csv('/content/drive/MyDrive/2000_random_rows.csv')
new_df

,Unnamed: 0,source,cos_sim,target
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн..."
...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч..."
1996,1225,— спросил он,0.568087,— спросил Сергей
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в..."


In [ ]:
pip install sentence-transformers

In [ ]:
import torch

In [ ]:
device = 'cpu'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en").to(device)

In [ ]:
def get_translation(text):
  encoding = tokenizer(text, return_tensors="pt", padding=True).to(device)
  with torch.no_grad():
    generated_ids = model.generate(**encoding, max_length=100)
  generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return generated_texts

In [ ]:
import logging

In [ ]:
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
%%time
new_df['eng_trans'] = new_df['source'].apply(get_translation)

CPU times: user 58min 50s, sys: 8.28 s, total: 58min 58s
Wall time: 59min 38s


In [ ]:
new_df

,Unnamed: 0,source,cos_sim,target,eng_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ..."
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha..."


In [ ]:
# Конвертация в csv-файл вместе с eng_trans
# new_df.to_csv("result_ENG.csv")

# Применение модели Helsinki-NLP/opus-mt-en-ru
# на параллельном корпусе парафраз RuAdapt
https://huggingface.co/Helsinki-NLP/opus-mt-en-ru

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_ENG.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ..."
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t..."
...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha..."


In [ ]:
pip install sentence-transformers

In [ ]:
import torch

In [ ]:
device = 'cpu'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru").to(device)

In [ ]:
def get_translation(text):
  encoding = tokenizer(text, return_tensors="pt", padding=True).to(device)
  with torch.no_grad():
    generated_ids = model.generate(**encoding, max_length=100)
  generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return generated_texts

In [ ]:
import logging

In [ ]:
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
%%time
final_df['rus_trans'] = final_df['eng_trans'].apply(get_translation)

CPU times: user 52min 29s, sys: 5.04 s, total: 52min 34s
Wall time: 53min 30s


In [ ]:
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
# Конвертация в csv-файл вместе с rus_trans
# final_df.to_csv("result_RUS.csv")

# Оценка результата моделей машинного обучения
# с помощью дистрибутивной модели rubert-base-cased-sentence

In [ ]:
pip install sentence-transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [ ]:
import gc
import multiprocessing as mp
from tqdm import tqdm

In [ ]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)

In [ ]:
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
%%time
DF_FILE = '/content/drive/MyDrive/result_RUS.csv'
# размер одного обрабатываемого кусочка датафрейма
CHUNKSIZE = 20

def process_frame(final_df):
  # функция принимает на вход датафрейм и возвращает косинусные подобия для каждого ряда
  similarities_4 = []
  for n, row in tqdm(final_df.iterrows()):
    s = row['source']
    o = row['rus_trans']
    tokens_s = tokenizer([s], return_tensors="pt", padding=True)
    tokens_o = tokenizer([o], return_tensors="pt", padding=True)
    out_s = model(**tokens_s)
    out_o = model(**tokens_o)
    vec_s = torch.mean(out_s.last_hidden_state, dim=1)
    vec_o = torch.mean(out_o.last_hidden_state, dim=1)
    cos_sim = cos(vec_s, vec_o)
    # item() превращает тензор во float
    similarities_4.append(cos_sim.item())
    # на всякий случай собираем ненужное при помощи garbage collector
    gc.collect()
  return similarities_4

# читаем файл по кусочкам заданного размера
reader = pd.read_csv(DF_FILE, chunksize=CHUNKSIZE)
# количество процессов, которые могут идти одновременно
pool = mp.Pool(4)

funclist_4 = []
for final_df_4 in reader:
  # открываем и обрабатываем каждый кусочек
  final_df.columns = ['Unnamed: 0.1', 'source', 'cos_sim', 'target', 'eng_trans', 'rus_trans']
  f = pool.apply_async(process_frame, [final_df_4])
  funclist_4.append(f)

result_4 = []
for f in funclist_4:
  # таймаут можно увеличить, если возникает ошибка
  sims = f.get(timeout=500)
  result_4.extend(sims)

20it [00:48,  2.40s/it]
20it [00:48,  2.44s/it]
20it [00:49,  2.49s/it]
20it [00:51,  2.57s/it]
20it [00:33,  1.67s/it]
20it [00:39,  1.97s/it]
20it [00:38,  1.94s/it]
20it [00:42,  2.11s/it]
20it [00:38,  1.91s/it]
20it [00:35,  1.78s/it]
20it [00:38,  1.91s/it]
20it [00:38,  1.93s/it]
20it [00:37,  1.86s/it]
20it [00:37,  1.86s/it]
20it [00:43,  2.19s/it]
20it [00:38,  1.95s/it]
20it [00:40,  2.03s/it]
20it [00:38,  1.94s/it]
20it [00:41,  2.06s/it]
20it [00:37,  1.89s/it]
20it [00:36,  1.84s/it]
20it [00:40,  2.00s/it]
20it [00:44,  2.23s/it]
20it [00:46,  2.30s/it]
20it [00:36,  1.82s/it]
20it [00:37,  1.85s/it]
20it [00:34,  1.74s/it]
20it [00:34,  1.74s/it]
20it [00:40,  2.01s/it]
20it [00:37,  1.88s/it]
20it [00:38,  1.90s/it]
20it [00:36,  1.82s/it]
20it [00:35,  1.75s/it]
20it [00:38,  1.93s/it]
20it [00:38,  1.93s/it]
20it [00:39,  1.96s/it]
20it [00:34,  1.72s/it]
20it [00:34,  1.75s/it]
20it [00:35,  1.78s/it]
20it [00:37,  1.87s/it]
20it [00:39,  1.98s/it]
20it [00:39,  2.

CPU times: user 10.9 s, sys: 1.3 s, total: 12.2 s
Wall time: 16min 15s


In [ ]:
result_4[:2000]

[0.8083122372627258,
 0.9787716865539551,
 0.8989248275756836,
 0.933532178401947,
 0.9552593231201172,
 0.9617211222648621,
 0.9093510508537292,
 0.9426599144935608,
 0.9111666679382324,
 0.909275233745575,
 0.9517305493354797,
 0.6320735216140747,
 0.9723461270332336,
 0.9834252595901489,
 0.9894473552703857,
 0.9883856177330017,
 0.9177829623222351,
 0.9999998807907104,
 0.8825123906135559,
 0.9955266118049622,
 0.9767388105392456,
 0.9128007888793945,
 0.8892170786857605,
 0.9080710411071777,
 0.9414768815040588,
 0.8812808990478516,
 0.9656636714935303,
 0.9076095819473267,
 0.9059170484542847,
 0.5992151498794556,
 0.9596168994903564,
 0.964814305305481,
 0.9712166786193848,
 0.8871825933456421,
 0.9484243392944336,
 0.9208242893218994,
 0.9409360885620117,
 0.709287166595459,
 0.8001143336296082,
 0.922631025314331,
 0.8635338544845581,
 0.8397420048713684,
 0.8248805999755859,
 0.8548059463500977,
 0.9596822261810303,
 0.8638037443161011,
 0.8543869853019714,
 0.971262097358703

In [ ]:
# Среднее значение
import numpy as np
np.mean(np.array(result_4))

0.9059282682538032

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(result_4))

0.07764217878851196

# METEOR
https://huggingface.co/spaces/evaluate-metric/meteor

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

meteor = load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_RUS.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
predictions = final_df['rus_trans'].tolist()

In [ ]:
references = final_df['source'].tolist()

In [ ]:
results = meteor.compute(predictions=predictions, references=references)
results

{'meteor': 0.48594159991872415}

# BLEU
https://huggingface.co/spaces/evaluate-metric/bleu

In [ ]:
pip install evaluate

In [ ]:
from evaluate import load

bleu = load("bleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_RUS.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
output = final_df['rus_trans'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['Моя дочь не моя дочь, она Алексея Александровича.', '"Да, все кончено, и я снова один", сказала она себе, и не снимая шляпы, она села на стул, стоя у камина.', 'По его приглашению оба старейшины сразу же пошли к нему и побежали с ним так же легко, как они бегали с няней, мисс Гул, или с матерью.', 'Всё это произошло за один момент... офицер, не помнив себя, бросился на неё; рядом с Настасией Филипповной она больше не была свистком; порядочный джентльмен среднего возраста уже имел время полностью повеситься, и джентльмен отошел в сторону и смеялся над мочой.', 'Когда я родился грешником, я не жил так... квартира большая, уютная, и у меня целая комната и кроватка.', 'Но, друг мой, ты не помнишь покойного Николая Львовича?', 'Я думаю, он стесняется... когда мы вдвоем с ним, он очень умный и нежный, и в обществе он грубый человек, братья.', 'Она больше не плачет... она добрая... слышала глухой, дальний выстрел.', "Он так ревновал к блестящему Чарльзу д'Эвре, что мечтал, что обойдет его!"

In [ ]:
bleu_results = bleu.compute(predictions=predictions, references=references)
print(bleu_results)

{'bleu': 0.22582853759392466, 'precisions': [0.5711836829565925, 0.2952063668332408, 0.17141166414622921, 0.10154657293497364], 'brevity_penalty': 0.9702352151347639, 'length_ratio': 0.9706695245078685, 'translation_length': 34418, 'reference_length': 35458}


# SacreBLEU
https://huggingface.co/spaces/evaluate-metric/sacrebleu

In [ ]:
pip install evaluate

In [ ]:
pip install sacrebleu

In [ ]:
from evaluate import load

sacrebleu = load("sacrebleu")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_RUS.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
output = final_df['rus_trans'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
init = []

for word in source:
  init.append(word.split("''"))

init

In [ ]:
predictions = output
print(predictions)

references = init
print(references)

['Моя дочь не моя дочь, она Алексея Александровича.', '"Да, все кончено, и я снова один", сказала она себе, и не снимая шляпы, она села на стул, стоя у камина.', 'По его приглашению оба старейшины сразу же пошли к нему и побежали с ним так же легко, как они бегали с няней, мисс Гул, или с матерью.', 'Всё это произошло за один момент... офицер, не помнив себя, бросился на неё; рядом с Настасией Филипповной она больше не была свистком; порядочный джентльмен среднего возраста уже имел время полностью повеситься, и джентльмен отошел в сторону и смеялся над мочой.', 'Когда я родился грешником, я не жил так... квартира большая, уютная, и у меня целая комната и кроватка.', 'Но, друг мой, ты не помнишь покойного Николая Львовича?', 'Я думаю, он стесняется... когда мы вдвоем с ним, он очень умный и нежный, и в обществе он грубый человек, братья.', 'Она больше не плачет... она добрая... слышала глухой, дальний выстрел.', "Он так ревновал к блестящему Чарльзу д'Эвре, что мечтал, что обойдет его!"

In [ ]:
sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
print(sacrebleu_results)
print(round(sacrebleu_results["score"], 1))

{'score': 22.582853759392464, 'counts': [19659, 9570, 5214, 2889], 'totals': [34418, 32418, 30418, 28450], 'precisions': [57.118368295659245, 29.520636683324078, 17.14116641462292, 10.154657293497364], 'bp': 0.9702352151347639, 'sys_len': 34418, 'ref_len': 35458}
22.6


# ROUGE
https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
pip install evaluate

In [ ]:
pip install rouge_score

In [ ]:
from evaluate import load

rouge = load("rouge")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_RUS.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
output = final_df['rus_trans'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['Моя дочь не моя дочь, она Алексея Александровича.', '"Да, все кончено, и я снова один", сказала она себе, и не снимая шляпы, она села на стул, стоя у камина.', 'По его приглашению оба старейшины сразу же пошли к нему и побежали с ним так же легко, как они бегали с няней, мисс Гул, или с матерью.', 'Всё это произошло за один момент... офицер, не помнив себя, бросился на неё; рядом с Настасией Филипповной она больше не была свистком; порядочный джентльмен среднего возраста уже имел время полностью повеситься, и джентльмен отошел в сторону и смеялся над мочой.', 'Когда я родился грешником, я не жил так... квартира большая, уютная, и у меня целая комната и кроватка.', 'Но, друг мой, ты не помнишь покойного Николая Львовича?', 'Я думаю, он стесняется... когда мы вдвоем с ним, он очень умный и нежный, и в обществе он грубый человек, братья.', 'Она больше не плачет... она добрая... слышала глухой, дальний выстрел.', "Он так ревновал к блестящему Чарльзу д'Эвре, что мечтал, что обойдет его!"

In [ ]:
rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: x.split())
rouge_results

{'rouge1': 0.4235135023903791,
 'rouge2': 0.20027903481276868,
 'rougeL': 0.4114053451812457,
 'rougeLsum': 0.4114648700494}

# BERTScore
https://huggingface.co/spaces/evaluate-metric/bertscore

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

In [ ]:
from evaluate import load

bertscore = load("bertscore")

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_RUS.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
output = final_df['rus_trans'].tolist()

In [ ]:
source = final_df['source'].tolist()

In [ ]:
predictions = output
print(predictions)

references = source
print(references)

['Моя дочь не моя дочь, она Алексея Александровича.', '"Да, все кончено, и я снова один", сказала она себе, и не снимая шляпы, она села на стул, стоя у камина.', 'По его приглашению оба старейшины сразу же пошли к нему и побежали с ним так же легко, как они бегали с няней, мисс Гул, или с матерью.', 'Всё это произошло за один момент... офицер, не помнив себя, бросился на неё; рядом с Настасией Филипповной она больше не была свистком; порядочный джентльмен среднего возраста уже имел время полностью повеситься, и джентльмен отошел в сторону и смеялся над мочой.', 'Когда я родился грешником, я не жил так... квартира большая, уютная, и у меня целая комната и кроватка.', 'Но, друг мой, ты не помнишь покойного Николая Львовича?', 'Я думаю, он стесняется... когда мы вдвоем с ним, он очень умный и нежный, и в обществе он грубый человек, братья.', 'Она больше не плачет... она добрая... слышала глухой, дальний выстрел.', "Он так ревновал к блестящему Чарльзу д'Эвре, что мечтал, что обойдет его!"

In [ ]:
bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
bertscore_results

{'precision': [0.8898560404777527,
  0.961042582988739,
  0.9523546695709229,
  0.9232494831085205,
  0.9278700947761536,
  0.9852360486984253,
  0.9381609559059143,
  0.9308850169181824,
  0.9417468905448914,
  0.9581632018089294,
  0.910759449005127,
  0.8567219972610474,
  0.948886513710022,
  0.983506977558136,
  0.9549219608306885,
  0.9676362872123718,
  0.9306263327598572,
  1.000000238418579,
  0.9254299998283386,
  0.9690696001052856,
  0.9404910206794739,
  0.8828209638595581,
  0.9394344687461853,
  0.9478850364685059,
  0.9636414647102356,
  0.945762574672699,
  0.9589203596115112,
  0.893599271774292,
  0.9514164328575134,
  0.8122736215591431,
  0.9486159682273865,
  0.9802955389022827,
  0.9439871311187744,
  0.7969021201133728,
  0.9415478110313416,
  0.919134795665741,
  0.9317250847816467,
  0.8913697004318237,
  0.9176074862480164,
  0.9339972734451294,
  0.9177709817886353,
  0.93354332447052,
  0.8476791977882385,
  0.9164903163909912,
  0.9568418264389038,
  0.858

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['precision']))

0.9350041653811931

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['recall']))

0.9298202035725117

In [ ]:
import numpy as np
np.mean(np.array(bertscore_results['f1']))

0.932273087322712

# ParaScore
https://github.com/shadowkiller33/ParaScore

In [ ]:
pip install parascore==1.0.5

In [ ]:
from parascore import ParaScorer

In [ ]:
scorer = ParaScorer(lang="ru", model_type='bert-base-uncased')

In [ ]:
import pandas as pd
final_df = pd.read_csv('/content/drive/MyDrive/result_RUS.csv').drop("Unnamed: 0", axis=1)
final_df

,Unnamed: 0.1,source,cos_sim,target,eng_trans,rus_trans
0,15502,"Моя дочь по закону – не моя дочь, а Каренина.",0.679270,Но закон и все условия нашего положения таковы...,"My daughter isn't my daughter, she's Karenina.","Моя дочь не моя дочь, она Алексея Александровича."
1,23216,"«Да, все это кончено, и я опять одна», – сказа...",0.558342,"«Да, всё кончено, и я опять одна», — сказала о...","""Yeah, it's over, and I'm alone again,"" she sa...","""Да, все кончено, и я снова один"", сказала она..."
2,20768,На приглашение его два старшие тотчас же соско...,0.802290,На приглашение его двое старших тотчас же подо...,"At his invitation, the two elders immediately ...",По его приглашению оба старейшины сразу же пош...
3,1059,"Все это произошло в одно мгновение… Офицер, не...",0.331989,"Офицер, не помня себя, бросился на неё.","All this happened in one moment... an officer,...","Всё это произошло за один момент... офицер, не..."
4,11600,"Отродясь я, грешница, так не жила… Квартира бо...",0.683107,"Квартира большая, казённая, и мне цельная комн...","When I was born a sinner, I didn't live like t...","Когда я родился грешником, я не жил так... ква..."
...,...,...,...,...,...,...
1995,13103,"Но надо было ехать, и он по первому поезду, в ...",0.820885,"Но надо было ехать, и он первым поездом, в ноч...","But he had to go, and on the first train, the ...","Но он должен был пойти, и в первый поезд, в ту..."
1996,1225,— спросил он,0.568087,— спросил Сергей,"""and he asked","""и он спросил"
1997,13123,Ударит первым.,0.382794,Начнёт бой первым.,He's gonna hit me first.,Он ударит меня первым.
1998,16923,"— Конечно, конечно… Я разовью эту мысль, поста...",0.542537,"— Конечно, конечно… Я рад… Я постараюсь… Так в...","Of course, of course... I'm gonna build on tha...","Конечно, конечно... я буду опираться на эту мы..."


In [ ]:
cands = final_df['rus_trans'].tolist()
sources = final_df['source'].tolist()
refs = final_df['target'].tolist()

In [ ]:
parascore_results = scorer.base_score(cands, sources, refs, batch_size=16)
parascore_results

[0.7647628355026245,
 0.9071481069739984,
 0.8899989506736427,
 0.8444014954566955,
 0.8662535119056701,
 0.9295240810939244,
 0.865559594631195,
 0.8617773183997797,
 0.8935263191417239,
 0.8803616435187204,
 0.7629630732536316,
 0.7191231417655944,
 0.9011441040039062,
 0.9337575836010213,
 0.9083527493476867,
 0.9438306856155395,
 0.8842271733283996,
 0.95,
 0.8281130361557006,
 0.9012398876462664,
 0.8721394109725952,
 0.835733847618103,
 0.8822071121663464,
 0.8926412034034729,
 0.8940576370881528,
 0.875895032688063,
 0.9000116157531738,
 0.775909321308136,
 0.8668280410766601,
 0.6252550053596496,
 0.8901922232226321,
 0.9437913467113237,
 0.9007190036773681,
 0.6330269145965576,
 0.8836898374557495,
 0.8535015749931335,
 0.8436352181434631,
 0.7630520629882812,
 0.9689448642730712,
 0.8756260658132619,
 0.8290428686141967,
 0.8713010120391845,
 0.6738292145729065,
 0.8227903413772583,
 0.8987122344970703,
 0.7343315052986145,
 0.8306957650184631,
 0.8891723918914795,
 0.9706632

In [ ]:
import numpy as np
np.mean(np.array(parascore_results))

0.8589264893525261

In [ ]:
# Среднеквадратичное отклонение
np.std(np.array(parascore_results))

0.06632719856841832